In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
source = '/kaggle/input/siim-covid19-detection'
os.listdir(source)

In [ ]:
train_image_level = pd.read_csv(f'{source}/train_image_level.csv')
train_study_level = pd.read_csv(f'{source}/train_study_level.csv')
sample_submission = pd.read_csv(f'{source}/sample_submission.csv')

In [ ]:
train_image_level.info()

In [ ]:
train_study_level.info()

In [ ]:
sample_submission.info()

In [ ]:
train_image_level.head()

In [ ]:
train_study_level.head()

In [ ]:
sample_submission.head()

In [ ]:
train_image_level.shape

In [ ]:
train_study_level.shape

In [ ]:
sample_submission.shape

In [ ]:
train_study_level.columns

In [ ]:
train_image_level.columns

In [ ]:
train_image_level.isna()

In [ ]:
sample_submission.columns

In [ ]:
train_study_level.isna()

In [ ]:
sample_submission.isna()

In [ ]:
train_image_level['boxes']

In [ ]:
train_image_level['StudyInstanceUID']

In [ ]:
train_image_level['id']

In [ ]:
train_image_level['id'] = train_image_level.apply(lambda row: row.id.split('_')[0], axis=1)

In [ ]:
train_image_level['id']

In [ ]:
train_study_level['id']

In [ ]:
train_study_level['id'] = train_study_level.apply(lambda row: row.id.split('_')[0], axis=1)

In [ ]:
train_study_level['id']

In [ ]:
#Merge study csv to image csv.
train_study_level['StudyInstanceUID'] = train_study_level['id'].apply(lambda x: x.replace('_study', ''))
train_image_level = train_image_level.merge(train_study_level, on='StudyInstanceUID')
siim_covid = train_image_level.drop(['id_x','id_y'], axis=1)
siim_covid.head()

In [ ]:
import glob
from tqdm import tqdm
# Merge image path to the train_image_level dataframe.
train_dir = '/kaggle/input/siim-covid19-detection/train'
test_dir = '/kaggle/input/siim-covid19-detection/test'

training_paths = []

for sid in tqdm(siim_covid['StudyInstanceUID']):
    training_paths.append(glob.glob(os.path.join(train_dir,sid+"/*/*"))[0])

siim_covid['path'] = training_paths

siim_covid.head()

In [ ]:
siim_covid.shape

In [ ]:
siim_covid = siim_covid.dropna()

In [ ]:
siim_covid.shape

In [ ]:
siim_covid.head()

In [ ]:
siim_covid['Negative for Pneumonia'].value_counts()

In [ ]:
siim_covid['Typical Appearance'].value_counts()

In [ ]:
siim_covid['Indeterminate Appearance'].value_counts()

In [ ]:
siim_covid['Atypical Appearance'].value_counts()

In [ ]:
import pydicom 
from pydicom import dcmread
import matplotlib.pyplot as plt
%matplotlib inline
label_appearance = list(siim_covid.columns)[3:7]
row_img = 3
col_img = len(label_appearance)

c= 0
color = 'gray'
ds_pos = [0 for p in range(row_img)]

fig,ax = plt.subplots(row_img,col_img, figsize=(25,row_img*4))

for i in label_appearance:
    for r in range(row_img):
        link = siim_covid[siim_covid[i] == 1]['path'].iloc[r]
        ds_pos[r] = pydicom.dcmread(link).pixel_array
        ax[r,c].imshow(ds_pos[r], cmap=color)
        
      
    ax[0,c].set_title(i, font='Serif', fontsize=20)
    c +=1

In [ ]:
train_image_level['id']